<a href="https://colab.research.google.com/github/mattsmit4/DFScode/blob/main/_OwnersBox_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Install Packages & Start Timer

In [ ]:
pip install pulp

In [ ]:
# Importing packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pulp import *
import tensorflow as tf
from statistics import median
from google.colab import files
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.layers import Dropout
from keras.constraints import MaxNorm
from keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import itertools
import time


In [ ]:
# Record the start time
start_time = time.time()

###Update Rules for Optimizer

In [ ]:
# Current week in NFL
week = 18
# How many model iterations you want
iterations = 151
# Setting the floor of players to not include
floor = 0.2
# Setting a maximum for sum of Ownership as a percentage
Ownership = 0.84
# Set minimums for the projected points
NewProjectedPoints_rb_qb_wr = 8.93
NewProjectedPoints_te = 9.56

###Getting Historical Data and Shaping It

In [ ]:
# # Bringing in all the data
df_2013 = []
for i in range(1, 18):
    df = locals()[f'df{i}'] = pd.read_csv(fr"drive/MyDrive/Fantasy-Football-Coding/Fantasy-Data/OwnersBox/Combined/2013/combined-wk{i}.csv")
    df_2013.append(df)
df_2013 = pd.concat(df_2013)
df_2013 = df_2013.loc[df_2013.Position.isin(['QB', 'RB', 'WR', 'TE'])]

df_2014 = []
for i in range(1, 18):
    df = locals()[f'df{i}'] = pd.read_csv(fr"drive/MyDrive/Fantasy-Football-Coding/Fantasy-Data/OwnersBox/Combined/2014/combined-wk{i}.csv")
    df_2014.append(df)
df_2014 = pd.concat(df_2014)
df_2014 = df_2014.loc[df_2014.Position.isin(['QB', 'RB', 'WR', 'TE'])]

df_2015 = []
for i in range(1, 18):
    df = locals()[f'df{i}'] = pd.read_csv(fr"drive/MyDrive/Fantasy-Football-Coding/Fantasy-Data/OwnersBox/Combined/2015/combined-wk{i}.csv")
    df_2015.append(df)
df_2015 = pd.concat(df_2015)
df_2015 = df_2015.loc[df_2015.Position.isin(['QB', 'RB', 'WR', 'TE'])]

df_2016 = []
for i in range(1, 18):
    df = locals()[f'df{i}'] = pd.read_csv(fr"drive/MyDrive/Fantasy-Football-Coding/Fantasy-Data/OwnersBox/Combined/2016/combined-wk{i}.csv")
    df_2016.append(df)
df_2016 = pd.concat(df_2016)
df_2016 = df_2016.loc[df_2016.Position.isin(['QB', 'RB', 'WR', 'TE'])]

df_2017 = []
for i in range(1, 18):
    df = locals()[f'df{i}'] = pd.read_csv(fr"drive/MyDrive/Fantasy-Football-Coding/Fantasy-Data/OwnersBox/Combined/2017/combined-wk{i}.csv")
    df_2017.append(df)
df_2017 = pd.concat(df_2017)
df_2017 = df_2017.loc[df_2017.Position.isin(['QB', 'RB', 'WR', 'TE'])]

df_2018 = []
for i in range(1, 18):
    df = locals()[f'df{i}'] = pd.read_csv(fr"drive/MyDrive/Fantasy-Football-Coding/Fantasy-Data/OwnersBox/Combined/2018/combined-wk{i}.csv")
    df_2018.append(df)
df_2018 = pd.concat(df_2018)
df_2018 = df_2018.loc[df_2018.Position.isin(['QB', 'RB', 'WR', 'TE'])]

df_2019 = []
for i in range(1, 18):
    df = locals()[f'df{i}'] = pd.read_csv(fr"drive/MyDrive/Fantasy-Football-Coding/Fantasy-Data/OwnersBox/Combined/2019/combined-wk{i}.csv")
    df_2019.append(df)
df_2019 = pd.concat(df_2019)
df_2019 = df_2019.loc[df_2019.Position.isin(['QB', 'RB', 'WR', 'TE'])]

df_2020 = []
for i in range(1, 18):
    df = locals()[f'df{i}'] = pd.read_csv(fr"drive/MyDrive/Fantasy-Football-Coding/Fantasy-Data/OwnersBox/Combined/2020/combined-wk{i}.csv")
    df_2020.append(df)
df_2020 = pd.concat(df_2020)
df_2020 = df_2020.loc[df_2020.Position.isin(['QB', 'RB', 'WR', 'TE'])]

df_2021 = []
for i in range(1, 19):
    df = locals()[f'df{i}'] = pd.read_csv(fr"drive/MyDrive/Fantasy-Football-Coding/Fantasy-Data/OwnersBox/Combined/2021/combined-wk{i}.csv")
    df_2021.append(df)
df_2021 = pd.concat(df_2021)
df_2021 = df_2021.loc[df_2021.Position.isin(['QB', 'RB', 'WR', 'TE'])]

df_2022 = []
for i in range(1, 19):
    df = locals()[f'df{i}'] = pd.read_csv(fr"drive/MyDrive/Fantasy-Football-Coding/Fantasy-Data/OwnersBox/Combined/2022/combined-wk{i}.csv")
    df_2022.append(df)
df_2022 = pd.concat(df_2022)
df_2022 = df_2022.loc[df_2022.Position.isin(['QB', 'RB', 'WR', 'TE'])]

df_2023 = []
for i in range(1, week):
    df = locals()[f'df{i}'] = pd.read_csv(fr"drive/MyDrive/Fantasy-Football-Coding/Fantasy-Data/OwnersBox/Combined/2023/combined-wk{i}.csv")
    df_2023.append(df)
df_2023 = pd.concat(df_2023)
df_2023 = df_2023.loc[df_2023.Position.isin(['QB', 'RB', 'WR', 'TE'])]

# Combining all the yearly data frames into one
df = pd.concat([df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023])
df.shape

# Fill NaN values with 0.0
df = df.fillna(0.0)
# Resetting the index
df = df.reset_index(drop=True)

# Lists to Define the positions you want to filter for each group
positions_rb_qb_wr = ['RB', 'QB', 'WR']
positions_te = ['TE']

# This keeps all the data where the Projected Points are greather than or equal to what the variable is
# Create a condition based on Tight Ends
is_te = df['Position'] == 'TE'
# Filter the DataFrame based on the conditions
df.loc[is_te, 'Keep'] = (df['NewProjectedPoints'] >= NewProjectedPoints_te)
df.loc[~is_te, 'Keep'] = (df['NewProjectedPoints'] >= NewProjectedPoints_rb_qb_wr)

# Drop rows where 'Keep' is False
df = df[df['Keep']]

# Getting the Home and Away data from Categorical to Numbers
dummies = pd.get_dummies(df.HomeAway)

# Concatenate the dummy dataframe and the main one
df = pd.concat([df, dummies], axis='columns')
# Drop the 'Keep' column
df = df.drop(columns='Keep')

###Train & Test Split and Training the Model

In [ ]:
# Splitting the data into features and the label
X = df[['NewProjectedPoints', 'Home', 'OverUnder', 'Trend']].values
y = df[['NewActualPoints']].values

# Define hyperparamters to keep each iteration the same seed
random_seeds = []
for rs in range(1, iterations):
  random_seeds.append(rs)

# Lists to store performance metrics
val_loss_scores = []
best_model_paths = []
mse_loss_scores = []

# Split the data into training and testing and saving the best loss rate for each iteration of the model
for seed in random_seeds:
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, shuffle=True)

  # Scale both the train and test, otherwise the model will fail
  # Standardize features by removing the mean and scaling to unit variance
  scaler = StandardScaler()
  scaler.fit(X_train)

  # The new variables are being scaled based on the standardizing features that happened in X_train
  X_train_scaled = scaler.transform(X_train)
  X_test_scaled = scaler.transform(X_test)

  # Define the model
  model = Sequential()
  # First hidden layer number of neurons and number of inputs, with the type of activation
  model.add(Dense(6, input_shape=(4,), activation='relu'))
  # Second Hidden Layer
  model.add(Dense(6, activation='relu'))
  # Output layer
  model.add(Dense(1, activation='linear'))


  # Compile the model (which defines the loss function, the optimizer, and the metrics)
  model.compile(tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')
  model.summary

  # Save the best model
  checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

  # Fitting the model
  history = model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=11, callbacks=[checkpoint])

  # Saving the best model validation losses
  best_val_loss = min(history.history['val_loss'])
  val_loss_scores.append(best_val_loss)
  best_model = load_model('best_model.h5')
  best_model_paths.append(best_model)


  # Using the best model on the Test data now
  mse_neural = best_model.evaluate(X_test_scaled, y_test)
  print('')
  print("Iteration:", seed )
  print(f"Mean Squared Error: {mse_neural:.3f}")
  #print("Mean Squared Error from neural network: ", mse_neural)
  print(f"Best Validation Loss: {best_val_loss:.3f}")
  #print("Best Validation Loss: ", best_val_loss)
  print('')
  mse_loss_scores.append(mse_neural)

  # Plot the training and validation loss at each epoch
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(1, len(loss) + 1)
  # Does a line
  plt.plot(epochs, loss, 'y', label='Training Loss')
  # Does a dot
  #plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation Loss')
  plt.title("Training and Validation Loss")
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()
  print('')
  print('---------------------------------------------------------------------------')
  print('')


###Getting Top Ownership Lineup

In [ ]:
# Bring in the Data
df_ownership = pd.read_csv(f"drive/MyDrive/Fantasy-Football-Coding/Fantasy-Data/OwnersBox/_Current/Current-wk{week}.csv")
# Fill NaN values with 0.0
df_ownership = df_ownership.fillna(0.0)
df_ownership = df_ownership[['Name', 'Position', 'Salary', 'Ownership']]
df_ownership = df_ownership[df_ownership['Salary'] != 0]

# Group players by position
availables = df_ownership.groupby(["Name", "Position", "Salary",'Ownership']).size().reset_index(name='Count')

# Define empty dictionaries to track the metrics for the lineup optimizer
salaries = {}
players = {}
ownerships = {}
lineups_dict = {}

# Group players by position and create dictionaries for key player attributes
for position in availables.Position.unique():
    available_for_position = availables[availables.Position == position]

    players[position] = list(available_for_position['Name'])
    salaries[position] = dict(zip(available_for_position['Name'], available_for_position['Salary']))
    ownerships[position] = dict(zip(available_for_position['Name'], available_for_position['Ownership']))

# Define lineup constraints and variables
positions_available = {
    "QB": 2,
    "RB": 2,
    "WR": 2,
    "TE": 1,
    "FLEX": 2
}

flexible_positions = ("RB", "WR", "TE")
salary_cap = 50000
variables = {position: LpVariable.dict(position, players[position], cat="Binary") for position in players}

# Create lineups
for lineup in range(1, 2):
    prob = LpProblem("Fantasy", LpMaximize)

    rewards = []
    costs = []
    number_of_selected_players = []


    for position, players in variables.items():
        for player, player_selected in players.items():
            costs += salaries[position][player] * player_selected
            rewards += ownerships[position][player] * player_selected  # Use ownership instead of points
            number_of_selected_players += player_selected


        if position not in flexible_positions:
            prob += lpSum(players.values()) == positions_available[position]
        else:
            prob += lpSum(players.values()) >= positions_available[position]
            prob += lpSum(players.values()) <= positions_available[position] + positions_available["FLEX"]

    prob += lpSum(number_of_selected_players) == sum(positions_available.values())
    prob += lpSum(rewards)
    prob += lpSum(costs) <= salary_cap


    num = 0.001
    if not lineup == 1:
        prob += (lpSum(rewards) <= total_ownership - num)  # Update to use total ownership
    prob.solve()

    ownership = str(prob.objective)  # Use ownership instead of total score

    lineupList = []

    for v in prob.variables():
        ownership = ownership.replace(v.name, str(v.varValue))
        if v.varValue != 0:
            lineupList.append(v.name)

    total_ownership = eval(ownership)  # Update to use ownership
    lineupList.append(total_ownership)

    lineups_dict.update({lineup: lineupList})

# Create a DataFrame from the lineup data
df = pd.DataFrame(lineups_dict).T
newcols = ['QB', 'SUPER', 'RB1', 'RB2', 'FLEXorTE', 'TEorFLEX', 'FLEX2', 'WR1', 'WR2', 'Ownership']  # Update column name

df.columns = newcols

removeKeys = ['QB_', 'RB_', 'TE_', 'WR_']

for pos in newcols:
    for removeKey in removeKeys:
        df[pos] = df[pos].apply(str).str.replace(removeKey, "")
    df[pos] = df[pos].apply(str).str.replace("_", " ")

df['Ownership'] = df['Ownership'].astype(float).round(2).apply(lambda x: f'{x:.2f}')  # Format ownership as desired

first_row = df.head(1)
# Extract the 'Ownership' column from the first row
max_ownership = float(first_row['Ownership'].values[0])
new_ownership = max_ownership * Ownership

print(first_row)
print("")
print(new_ownership)

###Bringing in Upcoming Weeks Data

In [ ]:
# Bring in the data
df_test = pd.read_csv(f"drive/MyDrive/Fantasy-Football-Coding/Fantasy-Data/OwnersBox/_Current/Current-wk{week}.csv")

# Fill NaN values with 0.0
df_test = df_test.fillna(0.0)
# Get rid of low floor players
df_test = df_test[df_test['Floor'] >= floor]


# Create a condition based on 'Position'
is_te_test = df_test['Position'] == 'TE'

# Filter the DataFrame based on the condition
df_test.loc[is_te_test, 'Keep'] = (df_test['NewProjectedPoints'] >= NewProjectedPoints_te)
df_test.loc[~is_te_test, 'Keep'] = (df_test['NewProjectedPoints'] >= NewProjectedPoints_rb_qb_wr)

# Drop rows where 'Keep' is False
df_test = df_test[df_test['Keep']]

# Getting the Home and Away data from Categorical to Numbers
dummies_test = pd.get_dummies(df_test.HomeAway)

# Concatenate the dummy dataframe and the main one
df_test = pd.concat([df_test, dummies_test], axis='columns')

# Drop the 'Keep' column
df_test = df_test.drop(columns='Keep')

# Splitting the data into features and picking what features I want to include
X = df_test[['NewProjectedPoints', 'Home', 'OverUnder', 'Trend']].values

# Standardize the features so it matches the type of numbers that the model was trained on
scaler.fit(X)

# The new variables are being scaled based on the standardizing features that happened in X_train
X_scaled = scaler.transform(X)

###Setting the Constraints and Printing the Lineups

In [ ]:
# To set the minimum so that any errors in the training process don't get into results
std_dev = np.std(val_loss_scores)
loss_threshold = median(val_loss_scores) + std_dev + 3

df_list = []

for index, value in enumerate(val_loss_scores):
  if value <= loss_threshold:
    each_model = best_model_paths[index]
    formatted_value = "{:.2f}".format(value)
    print("Index:", index, "Value:", formatted_value)
    y_points = each_model.predict(X_scaled)

    # Add the new column into the dataframe
    df_test = df_test.assign(NNProjectedPoints=y_points)

    # Changing the NewProjectedPoints column to round to 3 decimal places
    decimals = 3
    df_test['NNProjectedPoints'] = df_test['NNProjectedPoints'].round(decimals)

    # Creating a DataFrame with just the listed columns
    df_test = df_test[['Name', 'Position', 'NNProjectedPoints', 'Salary', 'Team', 'Ownership', 'Trend']]
    # Keeping the rows that have a salary greater than 1
    df_test = df_test[df_test['Salary'] > 1]

    # This is regrouping the dataframe so the players are grouped by Position
    availables = df_test.groupby(["Position", "Name", "NNProjectedPoints", "Salary", 'Team', 'Ownership', 'Trend']).size().reset_index(name='Count')

    # Define empty dictionaries to track the metrics for the lineup optimizer
    salaries = {}
    points = {}
    players = {}
    teams = {}
    ownerships = {}
    trends = {}
    lineups_dict = {}


    # Group players by position
    for position in availables.Position.unique():
        available_for_position = availables[availables.Position == position]

        players[position] = list(available_for_position['Name'])
        salaries[position] = dict(zip(available_for_position['Name'], available_for_position['Salary']))
        points[position] = dict(zip(available_for_position['Name'], available_for_position['NNProjectedPoints']))
        teams[position] = dict(zip(available_for_position['Name'], available_for_position['Team']))
        ownerships[position] = dict(zip(available_for_position['Name'], available_for_position['Ownership']))
        trends[position] = dict(zip(available_for_position['Name'], available_for_position['Trend']))

    # This defines another dictionary that has the positions and lineup.
    positions_available = {
        "QB": 2,
        "RB": 2,
        "WR": 2,
        "TE": 1,
        "FLEX": 2
    }

    flexible_positions = ("RB", "WR", "TE")
    salary_cap = 50000

    # Create variables to track player selections
    variables = {position: LpVariable.dict(position, players[position], cat="Binary")
                  for position in players}

    # Extract unique team names from the DataFrame
    all_teams = df_test["Team"].unique().tolist()

    # Create a dictionary to track the number of players selected from each team
    team_selection_count = {team: lpSum(variables[position][(player, team)] for position in players for player in players[position] if player[1] == team)
                          for team in all_teams}

    # Iterate to create multiple lineups
    for lineup in range(1, 11):
      #The "problem" is we want to have the maximum salary for each lineup.
      prob = LpProblem("Fantasy", LpMaximize)

      # Creating an empty list to track
      rewards = []
      costs = []
      number_of_selected_players = []
      ownership_projections = []
      trends_ = []

      # Create a dictionary to track the team of each selected player
      selected_team = {}


      for position, players in variables.items():
        for player, player_selected in players.items():
          # If this player is selected, he will add salaries and points
            costs += salaries[position][player] * player_selected
            rewards += points[position][player] * player_selected
            number_of_selected_players += player_selected
            ownership_projections += ownerships[position][player] * player_selected
            trends_ += trends[position][player] * player_selected

        if position not in flexible_positions:
            prob += lpSum(players.values()) == positions_available[position]  # Filling in the QB
        else:
              prob += lpSum(players.values()) >= positions_available[position]  # Filling in the RB, WR, TE
              prob += lpSum(players.values()) <= positions_available[position] + positions_available["FLEX"]  # Filling in the FLEX


      # Create the same positional player cannot be on the same team
      for position, players in variables.items():
        for player, player_selected in players.items():
           if player_selected == 1:
              team = teams[position][player]
              players_from_same_team = [p for p in players if teams[position][p] == team]
              prob += lpSum(variables[position][p] for p in players_from_same_team) <= 1


      # This sums up the points and sums up the salary while making sure the lineup salary is less than or equal to the 50,000
      # In total, we need exactly `positions_available` players
      prob += lpSum(number_of_selected_players) == sum(positions_available.values())
      prob += lpSum(rewards)
      prob += lpSum(costs) <= salary_cap
      prob += lpSum(ownership_projections) <= new_ownership


      num = 0.001
      # If it's not the first lineup
      if not lineup == 1:
        # Make sure the rewards (points) is less than or equal too the total of the previous lineup, minus 0.01
        # That is too ensure you don't get the same lineup each time becasue the max points of the lineup caps out
        # at just below the previous total projections. This ensures varied lineups, and then solve
        prob += (lpSum(rewards) <= total_score - num)
      prob.solve()

      # Getting the strings for the lineup and the constraints (which is same position) when starting the new lineup.
      score = str(prob.objective)
      constraints = [str(const) for const in prob.constraints.values()]

      # Creating an empty list for storing the lineup
      lineupList = []

      # This will create a list of every player in the dataset with a variable of 1 or 0, depending the player
      # has been chosen for the lineup. This loops through the list to fill in if the player has been chosen or not.
      for v in prob.variables():
          score = score.replace(v.name, str(v.varValue))
          if v.varValue != 0:
            lineupList.append(v.name)


      # # Finally we just have to save the total_score variable for the lineup.
      total_score = eval(score)
      lineupList.append(total_score)

      # Calculate and display the sum of ownership percentages for the selected players
      total_ownership = sum(
          ownerships[position][player] * v.varValue
          for position, players in variables.items()
          for player, v in players.items()
          if v.varValue == 1)

      total_trends = sum(
          trends[position][player] * v.varValue
          for position, players in variables.items()
          for player, v in players.items()
          if v.varValue == 1

      )

      # Add the total ownership to the lineupList
      lineupList.append(total_ownership)
      lineupList.append(total_trends)


      # # Storing all of the lineups in a dictionary
      lineups_dict.update({lineup: lineupList})

    # Turning the dictionary into a DataFrame and Transpose
    df = pd.DataFrame(lineups_dict).T

    # Creating a list for what the rename the column names
    newcols = ['QB', 'SUPER', 'RB1', 'RB2', 'FLEXorTE', 'TEorFLEX', 'FLEX2', 'WR1', 'WR2', 'Total_Score', 'Ownership %', 'Trend']

    # Adding that column name list into the DataFrame
    df.columns = newcols

    # Creating a list of what values to remove in each particular cell
    removeKeys = ['QB_', 'RB_', 'TE_', 'WR_']
    # For each position in each column, when the player has the position and _ in front of it, remove that and replace it with nothing
    for pos in newcols:
      for removeKey in removeKeys:
          df[pos] = df[pos].apply(str).str.replace(removeKey,"")
      df[pos] = df[pos].apply(str).str.replace("_", " ")

    # Changing the columns to round to 2 decimal places
    df['Total_Score'] = df['Total_Score'].astype(float).round(2).apply(lambda x: f'{x:.2f}')
    df['Ownership %'] = df['Ownership %'].astype(float).round(2).apply(lambda x: f'{x:.2f}')

    # Showing what the top lineups are
    df_list.append(df)

###End Timer

In [ ]:
# Record the end time
end_time = time.time()
# Calculate the elapsed time in seconds
elapsed_time = end_time - start_time
# Convert elapsed time to minutes
elapsed_minutes = elapsed_time / 60
print(f"Total time taken: {elapsed_minutes:.0f} minutes")

Total time taken: 66 minutes


###Viewing the Top Lineups

In [ ]:
# Create the df from the appended list
result_df = pd.concat(df_list, ignore_index=True)

# Group the lineups based on 'Ownership' and 'Trend' and add a 'Group_Count' column
result_df['Count'] = result_df.groupby(['Ownership %', 'Trend']).cumcount() + 1

# Rename the 'Group_Count' column to 'Group_Count / Iterations'
result_df = result_df.rename(columns={'Count': f'Count / {iterations -1}'})

# Sort the DataFrame by 'Group_Count' in descending order
result_df = result_df.sort_values(by=f'Count / {iterations -1}', ascending=False).reset_index(drop=True)

# Remove duplicates based on 'Ownership' and 'Trend'
result_df = result_df.drop_duplicates(subset=['Ownership %', 'Trend'], keep='first')

# Convert 'Trend' column to numeric type
result_df['Trend'] = pd.to_numeric(result_df['Trend'], errors='coerce')

# Calculate adjusted 'Trend' values
result_df['Trend'] = round(result_df['Trend'] / 9, 2)

# Convert 'Trend' column to numeric type
result_df['Ownership %'] = pd.to_numeric(result_df['Ownership %'], errors='coerce')

# Getting the Ownership as a % of max
result_df['Ownership %'] = round(result_df['Ownership %'] / float(max_ownership), 2)

# Reset the index of the DataFrame
result_df = result_df.reset_index(drop=True)

# Start the index at 1 instead of 0
result_df.index = result_df.index + 1

result_df.head(5)